In [1]:
%%shell
git clone --recurse-submodules https://github.com/castorini/pyserini.git
cd pyserini/tools && cd eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../..
pip install pyserini faiss-cpu -qq
pip install transformers

Cloning into 'pyserini'...
remote: Enumerating objects: 7659, done.
remote: Counting objects: 100% (7659/7659), done.
remote: Compressing objects: 100% (2081/2081), done.
remote: Total 7659 (delta 5605), reused 7480 (delta 5539), pack-reused 0
Receiving objects: 100% (7659/7659), 4.40 MiB | 19.26 MiB/s, done.
Resolving deltas: 100% (5605/5605), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 806, done.        
remote: Counting objects: 100% (563/563), done.        
remote: Compressing objects: 100% (482/482), done.        
remote: Total 806 (delta 106), reused 527 (delta 80), pack-reused 243        
Receiving objects: 100% (806/806), 139.32 MiB | 6.54 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Submodule path 'tools': checked out '7b84f773225b5973b4533dfa0aa18653409a6146'
trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eva

In [2]:
import json
from tqdm.auto import tqdm

In [4]:
# Topics with all queries
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
!gzip -dc msmarco-test2019-queries.tsv.gz > queries.tsv

--2023-06-28 21:04:16--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4276 (4.2K) [application/x-gzip]
Saving to: ‘msmarco-test2019-queries.tsv.gz.1’

msmarco-test2019-qu 100%[===================>]   4.18K  --.-KB/s    in 0s      

2023-06-28 21:04:17 (701 MB/s) - ‘msmarco-test2019-queries.tsv.gz.1’ saved [4276/4276]



In [14]:
# Qrels with all relevances
!wget https://trec.nist.gov/data/deep/2019qrels-pass.txt

--2023-06-28 21:22:46--  https://trec.nist.gov/data/deep/2019qrels-pass.txt
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.175, 2610:20:6005:13::19
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187092 (183K) [text/plain]
Saving to: ‘2019qrels-pass.txt.1’

2019qrels-pass.txt. 100%[===================>] 182.71K   538KB/s    in 0.3s    

2023-06-28 21:22:47 (538 KB/s) - ‘2019qrels-pass.txt.1’ saved [187092/187092]



In [15]:
topics = []

with open("queries.tsv", "r") as f:
  for line in f:
    id, query = line.split("\t")
    topics.append((id, query))

with open("qrels.tsv", "w") as fout:
  with open("2019qrels-pass.txt", "r") as fin:
    for idx, line in enumerate(fin):
      if idx != 0:
        qid, _, doc_id, relevance = line.strip().split()
        fout.write(f"{qid}\t0\t{doc_id}\t{relevance}\n")

def get_run_bm25(path, topics, top_k, searcher):
    with open(path, 'w') as f:
        for id, query in tqdm(topics, desc="Running queries"):
            hits = searcher.search(query, top_k)
            for idx, hit in enumerate(hits):
              f.write(f"{id}\tQ0\t{hit.docid}\t{idx+1}\t{hit.score}\tpedro\n")

In [16]:
from pyserini.search.lucene import LuceneSearcher

In [17]:
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
get_run_bm25('original-bm25.tsv', topics, 100, searcher)

lucene-index.msmarco-v1-passage.20221004.252b5e.tar.gz: 2.02GB [01:11, 30.3MB/s]                            


Running queries:   0%|          | 0/200 [00:00<?, ?it/s]

In [18]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -l 2 qrels.tsv original-bm25.tsv

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 4.19MB/s]                
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-l', '2', 'qrels.tsv', 'original-bm25.tsv']
Results:
ndcg_cut_10           	all	0.5058


In [19]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

In [20]:
pretrained_model_name = "pedrogengo/listwise_longt5_1k_msmarco"

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
model = LongT5ForConditionalGeneration.from_pretrained(pretrained_model_name)

In [21]:
_ = model.to("cuda")
_ = model.eval()

In [22]:
def generate_permutation_text(query, passages):
  text = ["You are an intelligent assistant that can rank passages based on their relevancy to the query.",
          f"The following are {len(passages)} passages, each indicated by number identifier []. I can rank them based on their relevance to query: {query}",]

  for idx, passage in enumerate(passages):
    text.append(f"[doc_{idx+1}] {passage}")

  text.append(f"The search query is: {query}")
  text.append(f"I will rank the {len(passages)} passages above based on their relevance to the search query. The passages will be listed in descending order using identifiers, and the most relevant passages should be listed first, and the output format should be [] > [] > etc, e.g., [doc_1] > [doc_2] > etc.")
  text.append(f"The ranking results of the {len(passages)} passages (only identifiers) is:")
  return "\n".join(text)

In [23]:
def generate_prediction(model, prompt):
    model_inputs = tokenizer([prompt], padding="longest", return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(model_inputs, max_length=200, do_sample=False)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
      output = [docid.strip()[1:-1] for docid in pred.split(">")]
      assert len(output) == 9
      for i in range(1,10):
        assert f"doc_{i}" in output
      return output
    except:
      print("returning bm25 prediction")
      return ["doc_9", "doc_8", "doc_7", "doc_6", "doc_5", "doc_4", "doc_3", "doc_2", "doc_1"]

In [24]:
def get_run_listwise(path, topics, top_k, stride, searcher, model):
    window = 9
    with open(path, 'w') as f:
        for id, query in tqdm(topics, desc="Running queries"):
            hits = searcher.search(query, top_k)
            hits = list(reversed(hits))
            reranked = [0] * top_k
            for idx in range(0, len(hits) - stride, stride):
                if idx + window > len(hits):
                  idx = len(hits) - window
                passages = [json.loads(hit.raw)["contents"] for hit in hits[idx:idx+window]]
                mapping = {f"doc_{i+1}": [hit.docid, hit.score] for i, hit in enumerate(hits[idx:idx+window])}
                input_text = generate_permutation_text(query, passages)
                prediction = generate_prediction(model, input_text)

                for j, pred in zip(range(idx, idx + window), reversed(prediction)):
                  reranked[top_k - j - 1] = mapping[pred]

                new_order = []
                for docid in reversed(prediction):
                  new_order.append(hits[idx:idx+window][int(docid[-1]) - 1])
                hits[idx:idx+window] = new_order
            for idx, hit in enumerate(reranked):
              f.write(f"{id}\tQ0\t{hit[0]}\t{idx+1}\t{top_k - idx}\tpedro\n")

In [25]:
get_run_listwise("listwise-longt5.tsv", topics, 100, 4, searcher, model)

Running queries:   0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [26]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [27]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -l 2 qrels.tsv listwise-longt5.tsv

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-l', '2', 'qrels.tsv', 'listwise-longt5.tsv']
Results:
ndcg_cut_10           	all	0.4593
